In [1]:
%load_ext autoreload
%autoreload 2

## 1. Imports and Setup

In [2]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append("../src")

from strategy import Strategy
from backtest import run_backtest

# 2. Load and clean data

In [3]:
# Load your BTC/USD hourly data
data_path = '../data/btc_hour.csv'
df = pd.read_csv(data_path)
df.head()

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,2011-11-28 00:00:00+00:00,2.499,2.461,2.480,3457.45,8571.74,2.497,direct,NaN
1,2011-11-28 01:00:00+00:00,2.498,2.480,2.497,681.46,1698.15,2.498,direct,NaN
2,2011-11-28 02:00:00+00:00,2.499,2.485,2.498,624.29,1559.13,2.487,direct,NaN
3,2011-11-28 03:00:00+00:00,2.498,2.442,2.487,2648.16,6519.05,2.442,direct,NaN
4,2011-11-28 04:00:00+00:00,2.500,2.440,2.442,4299.98,10667.19,2.462,direct,NaN


In [4]:
df

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,2011-11-28 00:00:00+00:00,2.499,2.461,2.480,3457.45,8.571740e+03,2.497,direct,NaN
1,2011-11-28 01:00:00+00:00,2.498,2.480,2.497,681.46,1.698150e+03,2.498,direct,NaN
2,2011-11-28 02:00:00+00:00,2.499,2.485,2.498,624.29,1.559130e+03,2.487,direct,NaN
3,2011-11-28 03:00:00+00:00,2.498,2.442,2.487,2648.16,6.519050e+03,2.442,direct,NaN
4,2011-11-28 04:00:00+00:00,2.500,2.440,2.442,4299.98,1.066719e+04,2.462,direct,NaN
...,...,...,...,...,...,...,...,...,...
118054,2025-05-16 22:00:00+00:00,103649.830,103450.360,103643.590,978.41,1.012609e+08,103551.320,direct,NaN
118055,2025-05-16 23:00:00+00:00,103718.440,103465.470,103551.320,312.40,3.235507e+07,103499.600,direct,NaN
118056,2025-05-17 00:00:00+00:00,103569.600,103141.310,103499.600,488.11,5.044067e+07,103388.020,direct,NaN
118057,2025-05-17 01:00:00+00:00,103388.090,102642.390,103388.020,669.72,6.891526e+07,102859.210,direct,NaN


In [5]:
# Data cleaning: keep only time, close, volumeto; cast time to datetime; rename volumeto to volume
df = df[['time', 'close', 'volumeto']].copy()
df['time'] = pd.to_datetime(df['time'])
df = df.rename(columns={'volumeto': 'volume'})
df.head()

,time,close,volume
0,2011-11-28 00:00:00+00:00,2.497,8571.74
1,2011-11-28 01:00:00+00:00,2.498,1698.15
2,2011-11-28 02:00:00+00:00,2.487,1559.13
3,2011-11-28 03:00:00+00:00,2.442,6519.05
4,2011-11-28 04:00:00+00:00,2.462,10667.19


In [6]:
df.tail()

,time,close,volume
118054,2025-05-16 22:00:00+00:00,103551.32,1.012609e+08
118055,2025-05-16 23:00:00+00:00,103499.60,3.235507e+07
118056,2025-05-17 00:00:00+00:00,103388.02,5.044067e+07
118057,2025-05-17 01:00:00+00:00,102859.21,6.891526e+07
118058,2025-05-17 02:00:00+00:00,103307.02,2.715254e+07


# 3. Define strategy

### Buy and Hold strategy

In [11]:
class BuyAndHoldStrategy(Strategy):
    def __init__(self, initial_capital=10000):
        super().__init__(initial_capital)
        self.has_bought = False

    def process_bar(self, bar):
        self.current_bar = bar

    def get_signal(self):
        if not self.has_bought:
            self.has_bought = True
            return 'buy'
        return 'hold'

### SMA Crossover Strategy

In [12]:
class SMACrossoverStrategy(Strategy):
    def __init__(self, initial_capital=10000, fast=20, slow=100):
        super().__init__(initial_capital)
        self.prices = []
        self.fast = fast
        self.slow = slow
        self.last_signal = 'hold'

    def process_bar(self, bar):
        self.current_bar = bar
        self.prices.append(bar['close'])
        if len(self.prices) < self.slow:
            self.last_signal = 'hold'
            return

        fast_ma = pd.Series(self.prices).rolling(self.fast).mean().iloc[-1]
        slow_ma = pd.Series(self.prices).rolling(self.slow).mean().iloc[-1]

        if fast_ma > slow_ma and self.position == 0:
            self.last_signal = 'buy'
        elif fast_ma < slow_ma and self.position == 1:
            self.last_signal = 'sell'
        else:
            self.last_signal = 'hold'

    def get_signal(self):
        return self.last_signal

# 4. Run Backtests

In [17]:
# Buy and Hold
bh_results = run_backtest(BuyAndHoldStrategy, data_path, initial_capital=10000)

# SMA Crossover
sma_results = run_backtest(
    lambda initial_capital: SMACrossoverStrategy(initial_capital, fast=20, slow=100),
    data_path,
    initial_capital=10000
)

BUY: time=2011-11-28 00:00:00+00:00, entry_idx=0, equity_curve_len=1
FINAL SELL: time=2012-01-08 15:00:00+00:00, exit_idx=1000, equity_curve_len=1001
REALIZED DD: entry_idx=0, exit_idx=1000, equity_len=1002
BUY: time=2011-12-02 03:00:00+00:00, entry_idx=99, equity_curve_len=100
SELL: time=2011-12-03 23:00:00+00:00, exit_idx=143, equity_curve_len=144
BUY: time=2011-12-06 12:00:00+00:00, entry_idx=204, equity_curve_len=205
SELL: time=2011-12-10 12:00:00+00:00, exit_idx=300, equity_curve_len=301
BUY: time=2011-12-10 15:00:00+00:00, entry_idx=303, equity_curve_len=304
SELL: time=2011-12-14 14:00:00+00:00, exit_idx=398, equity_curve_len=399
BUY: time=2011-12-16 17:00:00+00:00, entry_idx=449, equity_curve_len=450
SELL: time=2011-12-16 23:00:00+00:00, exit_idx=455, equity_curve_len=456
BUY: time=2011-12-17 02:00:00+00:00, entry_idx=458, equity_curve_len=459


<string>:9: RuntimeWarning: overflow encountered in scalar power


SELL: time=2011-12-23 23:00:00+00:00, exit_idx=623, equity_curve_len=624
BUY: time=2011-12-24 17:00:00+00:00, entry_idx=641, equity_curve_len=642
SELL: time=2011-12-27 05:00:00+00:00, exit_idx=701, equity_curve_len=702
BUY: time=2011-12-27 07:00:00+00:00, entry_idx=703, equity_curve_len=704
SELL: time=2011-12-27 08:00:00+00:00, exit_idx=704, equity_curve_len=705
BUY: time=2011-12-28 02:00:00+00:00, entry_idx=722, equity_curve_len=723
SELL: time=2012-01-04 04:00:00+00:00, exit_idx=892, equity_curve_len=893
BUY: time=2012-01-04 19:00:00+00:00, entry_idx=907, equity_curve_len=908
FINAL SELL: time=2012-01-08 15:00:00+00:00, exit_idx=1000, equity_curve_len=1001
REALIZED DD: entry_idx=99, exit_idx=143, equity_len=1002
REALIZED DD: entry_idx=204, exit_idx=300, equity_len=1002
REALIZED DD: entry_idx=303, exit_idx=398, equity_len=1002
REALIZED DD: entry_idx=449, exit_idx=455, equity_len=1002
REALIZED DD: entry_idx=458, exit_idx=623, equity_len=1002
REALIZED DD: entry_idx=641, exit_idx=701, equi

# 5. Compare Results

In [18]:
def print_metrics(name, results):
    print(f"--- {name} ---")
    print(f"Sharpe Ratio: {results['sharpe']:.2f}")
    print(f"Total Return: {results['total_return']*100:.2f}%")
    print(f"Annualized Return: {results['annualized_return']*100:.2f}%")
    print(f"Max Drawdown: {results['max_drawdown']*100:.2f}%")
    print(f"Number of Trades: {results['n_trades']}")
    print(f"Win Rate: {results['win_rate']*100:.2f}%")
    print()

print_metrics("Buy & Hold", bh_results)
print_metrics("SMA Crossover", sma_results)

--- Buy & Hold ---
Sharpe Ratio: 112.63
Total Return: 2148105454290794856466699068236800800503786496161255078823560462371714843863969847136109190854518075548105408478413010769041859615702698277607980437243159210344690781683449856.00%
Annualized Return: inf%
Max Drawdown: 6.21%
Number of Trades: 1
Win Rate: 100.00%

--- SMA Crossover ---
Sharpe Ratio: 64.25
Total Return: 603893229242880084061620011008.00%
Annualized Return: 131248065859484342859606158745341502353610048657544670722515784348052165211714971591423342213986802501605163109755122069405113302995558543962617148600243805430690676625306780440693846136810129777985428238519655103985119586917293540114824922398720.00%
Max Drawdown: 88.48%
Number of Trades: 9
Win Rate: 66.67%



In [20]:
bh_results

{'sharpe': np.float64(112.63461728173701),
 'total_return': np.float64(2.1481054542907947e+172),
 'n_trades': 1,
 'win_rate': 1.0,
 'max_drawdown': np.float64(0.06213248720942943),
 'annualized_return': np.float64(inf),
 'equity_curve': 0        1.000000e+04
 1        1.000000e+04
 2        1.000400e+04
 3        9.963941e+03
 4        9.744471e+03
             ...      
 997     3.225895e+174
 998     9.163505e+174
 999     2.620608e+175
 1000    7.502895e+175
 1001    2.148105e+176
 Length: 1002, dtype: float64,
 'trades': [{'entry': Timestamp('2011-11-28 00:00:00+0000', tz='UTC'),
   'entry_idx': 0,
   'entry_price': 2.497,
   'exit': Timestamp('2012-01-08 15:00:00+0000', tz='UTC'),
   'exit_idx': 1000,
   'exit_price': np.float64(7.149),
   'pnl': np.float64(1.3978159985117886e+176)}],
 'rolling_sharpe': 0         0.000000
 1         0.000000
 2         0.000000
 3         0.000000
 4         0.000000
            ...    
 997     189.858658
 998     189.849202
 999     189.830427
 

In [21]:
sma_results

{'sharpe': np.float64(64.25261454123307),
 'total_return': np.float64(6.038932292428801e+27),
 'n_trades': 9,
 'win_rate': 0.6666666666666666,
 'max_drawdown': np.float64(0.8848385052868633),
 'annualized_return': np.float64(1.3124806585948434e+243),
 'equity_curve': 0       1.000000e+04
 1       1.000000e+04
 2       1.000000e+04
 3       1.000000e+04
 4       1.000000e+04
             ...     
 997     1.983607e+31
 998     2.605504e+31
 999     3.445538e+31
 1000    4.561510e+31
 1001    6.038932e+31
 Length: 1002, dtype: float64,
 'trades': [{'entry': Timestamp('2011-12-02 03:00:00+0000', tz='UTC'),
   'entry_idx': 99,
   'entry_price': 3.113,
   'exit': Timestamp('2011-12-03 23:00:00+0000', tz='UTC'),
   'exit_idx': 143,
   'exit_price': 2.794,
   'pnl': -428.23313216177104},
  {'entry': Timestamp('2011-12-06 12:00:00+0000', tz='UTC'),
   'entry_idx': 204,
   'entry_price': 3.02,
   'exit': Timestamp('2011-12-10 12:00:00+0000', tz='UTC'),
   'exit_idx': 300,
   'exit_price': 2.941

---

# To do right now
 - Make our backtest more efficient / faster
 - Sort out the broken metrics